In [1]:
from src.pingers import ping_shops, ping_creative_and_performance, ping_creative
from src.database.session import db

base I am using: lebesgue-dev.cgviinyjedbs.eu-central-1.rds.amazonaws.com


In [2]:
df = ping_creative(db=db, shop_id=16038)

TypeError: CRUDShopifyOrder.query_aov() got multiple values for argument 'db'

In [6]:
df

,shop_id,name
0,36833384,larmanos.myshopify.com
1,46864466,smart-home-robots.myshopify.com
2,43739609,ultra-pro-international.myshopify.com
3,50592831,yourpetsclosetco.myshopify.com
4,36370064,macchiaco.myshopify.com
...,...,...
1035,12480682,terra-cottage.myshopify.com
1036,28299708,duty-box.myshopify.com
1037,40914710,melyssamonroy.com
1038,18600257,toni-blubblub-store.myshopify.com


In [2]:
query = session.query(FacebookAd.ad_id)

In [3]:
query[0]

('23848754996430295',)

In [4]:
type(query)

sqlalchemy.orm.query.Query